In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()    

In [2]:
ASSET_DIR = "../data/assets/with_direction"
# Asset root for rendering. You can change this if you want to use custom game assets.
ASSET_ROOT = "../data/assets/"

In [3]:
# Rendering and display
from grid_universe.renderer.texture import TextureRenderer
from IPython.display import display
# Default renderer used throughout the notebook unless overridden in a cell
renderer = TextureRenderer(resolution=240, asset_root=ASSET_ROOT)
renderer_large = TextureRenderer(resolution=480, asset_root=ASSET_ROOT)

In [4]:
import os
import numpy as np
import torch, torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from typing import List
from PIL import Image, ImageDraw
from IPython.display import display
from sklearn.model_selection import train_test_split
import time
import random

In [5]:
def get_model(num_classes: int) -> nn.Module:
    res = nn.Sequential(
        nn.Conv2d(3, 8, kernel_size=3, padding=1),
        nn.LeakyReLU(0.1),
        nn.BatchNorm2d(8),
        nn.MaxPool2d(2),

        nn.Flatten(),

        nn.Linear(1, 64),
        nn.LeakyReLU(0.1),
        nn.Linear(64, num_classes),
    )
    return res    

In [ ]:
def draw_direction_triangles_on_image(
    image: Image.Image, size: int, dx: int, dy: int, count: int
) -> Image.Image:
    """
    Draw 'count' filled triangles pointing (dx, dy) on the given RGBA image.
    Triangles are centered: the centroid of each triangle is symmetrically arranged
    around the image center. Spacing is between triangle centroids.
    """
    if count <= 0 or (dx, dy) == (0, 0):
        return image

    draw = ImageDraw.Draw(image)
    cx, cy = size // 2, size // 2

    # Triangle geometry (relative to size)
    tri_height = max(4, int(size * 0.16))
    tri_half_base = max(3, int(size * 0.10))
    spacing = max(2, int(size * 0.12))  # distance between triangle centroids

    # Axis-aligned direction and perpendicular
    ux, uy = dx, dy  # points toward the triangle tip
    px, py = -uy, ux  # perpendicular (for base width)

    # Offsets for centroids: 1 -> [0], 2 -> [-0.5s, +0.5s], 3 -> [-s, 0, +s], ...
    offsets = [(i - (count - 1) / 2.0) * spacing for i in range(count)]

    # For an isosceles triangle, the centroid lies 1/3 of the height from the base toward the tip.
    # If C is the centroid, then:
    #   tip = C + (2/3)*tri_height * u
    #   base_center = C - (1/3)*tri_height * u
    tip_offset = (2.0 / 3.0) * tri_height
    base_offset = (1.0 / 3.0) * tri_height

    for off in offsets:
        # Centroid position
        Cx = cx + int(round(ux * off))
        Cy = cy + int(round(uy * off))

        # Tip and base-center positions
        tip_x = int(round(Cx + ux * tip_offset))
        tip_y = int(round(Cy + uy * tip_offset))
        base_x = int(round(Cx - ux * base_offset))
        base_y = int(round(Cy - uy * base_offset))

        # Base vertices around base center along the perpendicular
        p1 = (tip_x, tip_y)
        p2 = (
            int(round(base_x + px * tri_half_base)),
            int(round(base_y + py * tri_half_base)),
        )
        p3 = (
            int(round(base_x - px * tri_half_base)),
            int(round(base_y - py * tri_half_base)),
        )

        draw.polygon([p1, p2, p3], fill=(255, 255, 255, 220), outline=(0, 0, 0, 220))

    return image

In [20]:
labels = ["no_direction", "right", "left", "down", "up"]
directions = [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]

class RandomDirection:
    def __init__(self):
        pass

    def __call__(self, image: Image.Image):
        random.seed(time.time())
        decision = random.randint(0, 4)

        if decision == 0:
            return image, 0

        dx, dy = directions[decision]
        new_image: Image.Image = draw_direction_triangles_on_image(image=image, size=image.size[0], dx=dx, dy=dy, count=1)       

        return new_image, decision


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, base_dataset, transform):
        self.base_dataset = base_dataset
        self.random_direction = RandomDirection()
        self.targets = base_dataset.targets
        self.transform = transform

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        image, _ = self.base_dataset[idx]
        new_image, new_label = self.random_direction(image)
        image = self.transform(new_image)
        return image, new_label


In [32]:
def get_augmentations():
    T = transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop((30, 30)),
    ])
    return T

In [33]:
ASSET_DIR = "../data/assets/with_direction"

base_dataset = ImageFolder(root=ASSET_DIR)
dataset = CustomDataset(base_dataset, transform=get_augmentations())

(image, label) = dataset[11]

print(labels[label])
display(image)

left
